In [30]:
import os
import tweepy as tw
import pandas as pd
import re
import nltk
import json
import numpy as np
from wordcloud import WordCloud,STOPWORDS
nltk.download('punkt')   
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize as nltk_wt
from nltk.stem import PorterStemmer
from textblob import TextBlob
from datetime import datetime,timedelta ,date
# from pythainlp.corpus import thai_stopwords
from pythainlp import word_tokenize as pyt_wt
import requests
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Teaw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Teaw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
consumer_key= 'xxxx'
consumer_secret= 'xxxx'
access_token= 'xxxx'
access_token_secret= 'xxxx'

In [33]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [34]:
class ClawTwitter:
    def __init__(self, keyw:str): 
        self.listdata = []
        self.keyword = keyw
        self.data = self.get_datatweet()
        self.metadata = {}
        self.setup_dir()
        self.load_metadata()

    def setup_dir(self):
        if not os.path.exists(f"./data/"): 
            os.mkdir(f"./data/")
        if not os.path.exists(f"./data/tweets/"): 
            os.mkdir(f"./data/tweets/")
            # print("crete success")
        if not os.path.exists('./data/metadata.json'):
            data = {'keyword' : {}}
            with open('./data/metadata.json', 'w', encoding="UTF-8") as file: #utf-16
                metafile = json.dumps(data, indent=4)
                file.write(metafile)
        
    def load_metadata(self):
        metadata_json = open('./data/metadata.json',encoding="UTF-8")
        self.metadata = json.load(metadata_json)

    def save_metadata(self):
        with open('./data/metadata.json', 'w', encoding="UTF-8") as file:
            metafile = json.dumps(self.metadata, indent=4)
            file.write(metafile)


    def connect(self):
        try:
            auth = tw.OAuthHandler(consumer_key, consumer_secret)
            auth.set_access_token(access_token, access_token_secret)
            api = tw.API(auth)
            return api
        except:
            print("Error")
            exit(1)

    def remove_url(txt):
        """Replace URLs found in a text string with nothing 
        (i.e. it will remove the URL from the string).

        Parameters
        ----------
        txt : string
            A text string that you want to parse and remove urls.

        Returns
        ----------
        The same txt string with url's removed.
        """

        return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

    def remove_url_th(txt):
        """Replace URLs found in a text string with nothing 
        (i.e. it will remove the URL from the string).

        Parameters
        ----------
        txt : string
            A text string that you want to parse and remove urls.

        Returns
        ----------
        The same txt string with url's removed.
        """

        return " ".join(re.sub("([^\u0E00-\u0E7Fa-zA-Z' ]|^'|'$|''|(\w+:\/\/\S+))", "", txt).split())

    def clean_text(self, text):
        text = text.lower()
        text = re.sub(r'(@[A-Za-z0-9_]+)', '', text)
        text = re.sub('http://\S+|https://\S+', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        text_tokens = nltk_wt(text)
        text = [word for word in text_tokens if not word in stopwords.words()]
        text = ' '.join(text)
        return text


    def clean_text_th(self, text):
        text = re.sub('http://\S+|https://\S+', '', text)
        url = "https://api.aiforthai.in.th/textcleansing"
        params = {f'text':{text}}
        headers = {
            'Apikey': "xxxx",
            }
        res = requests.request("GET", url, headers=headers, params=params)
        # print('test sen', res.json())
        return res.json()['cleansing_text']


    def stemmer(self, text):
        porter_text = PorterStemmer()
        token_words = nltk_wt(text)
        stem_word = []
        for word in token_words:
            stem_word.append(porter_text.stem(word))
        return " ".join(stem_word)


    def sentiment(self, text_en):
        sentiment_text = text_en.sentiment.polarity
        if  sentiment_text > 0:
            return 'positive'
        elif sentiment_text < 0:
            return 'negative'
        else:
            return 'neutral'


    def sentiment_th(self, text_th):
        url = "https://api.aiforthai.in.th/ssense"
        params = {'text':text_th}
        headers = {
            'Apikey': "xxxx"
            }
        res = requests.get(url, headers=headers, params=params)
        if res.json()['sentiment']['polarity'] == '':
            polarity = "neutral"
        else:
            polarity = res.json()['sentiment']['polarity']
        # print('hey', res.json())
        return polarity


    def get_datatweet(self):
        if not os.path.exists(f"./data/tweets/{self.keyword}"):
            os.makedirs(f"./data/tweets/{self.keyword}")
        api = self.connect()
        today = datetime.utcnow().date()
        today_str = today.strftime("%Y-%m-%d")
        tweets = tw.Cursor(api.search_tweets,
                    q=self.keyword + " -filter:retweets",
                    since= today_str).items(100)
                    # since= today_str).items(10)
                     
        tweet_list = []    
        for tweet in tweets:
            tweet_list.append(tweet)
        tweets = tweet_list

        data_tweet = [[self.keyword,
                        tweet.user.screen_name,
                        # tweet.lang,
                        tweet.user.location if tweet.user.location != '' else 'unknown',
                        tweet.created_at.replace(tzinfo = None) + timedelta(hours = 7),
                        ClawTwitter.remove_url(tweet.text) if tweet.lang == "en" else ClawTwitter.remove_url_th(tweet.text),
                        tweet.retweet_count,
                        tweet.favorite_count,
                        # self.count_for_data(self.data_hashtagcount()),
                        # self.sentiment(TextBlob(self.stemmer(self.clean_text(ClawTwitter.remove_url(tweet.text))))),
                        # self.sentiment(TextBlob(self.stemmer(ClawTwitter.remove_url(tweet.text)))),
                        self.sentiment(TextBlob(self.stemmer(self.clean_text(ClawTwitter.remove_url(tweet.text)))))
                                        if tweet.lang == "en" else self.sentiment_th(self.clean_text_th(tweet.text)),
                        # self.sentiment_th(tweet.text) if tweet.lang == "th"
                        #             else self.sentiment(TextBlob(self.stemmer(self.clean_text(ClawTwitter.remove_url(tweet.text))))),
                        f"https://twitter.com/twitter/statuses/{tweet.id}"] for tweet in tweets]

        return data_tweet


    def show_data(self, data_tweet:list):
        tweet_frame = pd.DataFrame(data=data_tweet,
                            # columns=['topic', 'user','language','location','post date', 'tweet','retweet','likes', 'sentiment','tweet link'])
                            # columns=['topic', 'user','location','post date', 'tweet','retweet','likes', 'count', 'sentiment','tweet link'])
                            columns=['topic', 'user','location','post date', 'tweet','retweet','likes', 'sentiment','tweet link'])
        
        # tweet_frame.to_excel(f"{self.keyword}.xlsx", engine="openpyxl", index=False)
        return tweet_frame


    def twitter_to_xlsx(self, tweets):
        hashtag = self.keyword
        today = date.today()
        today_str = today.strftime("%d%m%Y")
        # if not os.path.exists(f"./data/tweets/{hashtag}"):
        #     os.makedirs(f"./data/tweets/{hashtag}")
        if hashtag not in self.metadata['keyword'].keys():
            self.metadata['keyword'][hashtag] = {'date' : []}
        # tweets.to_excel(hashtag+"_"+today_str+".xlsx", engine="openpyxl", index=False)
        tweets.to_excel(f"./data/tweets/{hashtag}/{hashtag}_{today_str}.xlsx", engine="openpyxl", index=False)
        if today_str not in self.metadata['keyword'][hashtag]['date']:
            self.metadata['keyword'][hashtag]['date'].append(today_str)
            self.save_metadata()
    

    def data_hashtagcount(self):
        api = self.connect()
        today = datetime.utcnow().date()
        today_str = today.strftime("%Y-%m-%d")
        new_column = []
        for tweet in tw.Cursor(api.search_tweets,
                    q=self.keyword + " -filter:retweets",
                    since= today_str).items(100):
            entity_ht = tweet.entities.get('hashtags')
            hashtag = ""
            for i in range(0, len(entity_ht)):
                hashtag = hashtag + ",#" + entity_ht[i]['text']
            new_column.append([hashtag])
        data_frame = pd.DataFrame(data = new_column, columns=["hashtag"])
        # self.count_wordframe(data_frame)
        return data_frame
    

    def twitterhashtage_to_xlsx(self, datframe):
        hashtag = self.keyword
        today = date.today()
        today_str = today.strftime("%d%m%Y")
        datframe.to_excel(hashtag+"_"+"hashcount"+"_"+today_str+".xlsx", engine="openpyxl", index=False)


    def slash_tokenize(self,data):  
        result = data.split(",")
        return result


    # def count_wordframe(self, readfile):
    def count_wordframe(self):
        topic = self.keyword
        today = date.today()
        today_str = today.strftime("%d%m%Y")
        readfile = pd.read_excel(topic+"_"+"hashcount"+"_"+today_str+".xlsx")
        hastag_data = readfile["hashtag"].dropna()
        vectorizer = CountVectorizer(tokenizer = self.slash_tokenize)
        transformed_data = vectorizer.fit_transform(hastag_data)
        hash_tag_cnt_df = pd.DataFrame(columns = ['word', 'count']) 
        hash_tag_cnt_df['word'] = vectorizer.get_feature_names()
        hash_tag_cnt_df['count'] = np.ravel(transformed_data.sum(axis=0))
        tophash_tag = hash_tag_cnt_df.sort_values(by=['count'], ascending=False).head(20)
        return tophash_tag

    def twittercount_to_xlsx(self, datframe):
        hashtag = self.keyword
        today = date.today()
        today_str = today.strftime("%d%m%Y")
        datframe.to_excel(hashtag+"_"+"count"+"_"+today_str+".xlsx", engine="openpyxl", index=False)



    def tokenize_en(self, d):
        # print('hey', d)
        d = re.sub('[ก-๙]', '', d)
        d = re.sub('[0-9]', '', d)
        # print('test d', d)
        result = d.split(" ")
        result = list(filter(None, result))
        return result

    def count_word_en(self, dfr):
        new_text = []
        for txt in dfr["tweet"]:
            new_text.append(self.clean_text(txt))
        
        vectorizer = CountVectorizer(tokenizer=self.tokenize_en)
        transformed_data = vectorizer.fit_transform(new_text)
        keyword_df_en = pd.DataFrame(columns = ['word', 'count'])
        keyword_df_en['word'] = vectorizer.get_feature_names()
        keyword_df_en['count'] = np.ravel(transformed_data.sum(axis=0))
        topword_en = keyword_df_en.sort_values(by=['count'], ascending=False).head(20)

        return topword_en


In [35]:
# test = ClawTwitter("#REDVELVET", "th")
test = ClawTwitter("#DREAMCATCHER")

Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since


In [36]:
test.show_data(test.data)

,topic,user,location,post date,tweet,retweet,likes,sentiment,tweet link
0,#DREAMCATCHER,dcatcherbr,Desde 05/12/2016,2022-04-11 03:16:47,l INSTAGRAM Atualizao da SuA no story sualelbo...,2,3,neutral,https://twitter.com/twitter/statuses/151324965...
1,#DREAMCATCHER,deukaenthusiast,☣☠☣,2022-04-11 03:10:17,my friends just bullied me for being active on...,0,1,positive,https://twitter.com/twitter/statuses/151324802...
2,#DREAMCATCHER,deukaemobile,don't repost/claim as yours,2022-04-11 03:06:57,Maison 'MV Teaser' Desktop version hfdreamcatc...,15,63,neutral,https://twitter.com/twitter/statuses/151324718...
3,#DREAMCATCHER,yoohki_,she they # lesbian # 17,2022-04-11 03:03:05,Dreamcatcher Heardle 15 Dreamcatcher2ndAlbumAp...,0,0,neutral,https://twitter.com/twitter/statuses/151324620...
4,#DREAMCATCHER,YoohyeonHourly,unknown,2022-04-11 03:00:00,Dreamcatcher YOOHYEON hfdreamcatcher,1,4,neutral,https://twitter.com/twitter/statuses/151324543...
...,...,...,...,...,...,...,...,...,...
95,#DREAMCATCHER,JTP_KPOP,JTP ENTERTAINMENT,2022-04-11 00:11:45,Dreamcatcher 'MAISON' MV Teaser,0,1,neutral,https://twitter.com/twitter/statuses/151320309...
96,#DREAMCATCHER,conundrum329,unknown,2022-04-11 00:10:08,quote night Dreamcatcher life,0,0,neutral,https://twitter.com/twitter/statuses/151320268...
97,#DREAMCATCHER,mellowgona1,unknown,2022-04-11 00:08:47,Flat earthers are claiming this was staged and...,1,7,negative,https://twitter.com/twitter/statuses/151320234...
98,#DREAMCATCHER,caramelcream_MY,Dream World,2022-04-11 00:06:33,hfdreamcatcher Dreamcatcher ApocalypseSaveUs,1,5,neutral,https://twitter.com/twitter/statuses/151320178...


In [37]:
tweets_data = test.show_data(test.data)
test.twitter_to_xlsx(tweets_data)

In [38]:
tweets_data = test.show_data(test.data)
test.count_word_en(tweets_data)

C:\Users\Teaw\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,word,count
78,dreamcatcher,59
136,hfdreamcatcher,23
16,apocalypsesaveus,14
188,maison,10
131,heardle,9
83,dreamcatcherndalbumapocalypse,8
15,apocalypse,8
272,saveusdreamcatchercomeback,7
270,save,7
336,us,7


In [ ]:
# tweets_data = test.show_data(test.data)
# test.count_word_th(tweets_data)

In [ ]:
test.data_hashtagcount()

Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since


,hashtag
0,",#TXT,#HUENINGKAI,#TOMORROW_X_TOGEHTER,#휴닝카이,#..."
1,",#HUENINGKAI,#TXT,#TOMORROW_X_TOGETHER"
2,
3,",#YEONJUN"
4,",#TXT_IS_COMING,#TXT,#TOMORROW_X_TOGETHER"
...,...
95,
96,",#연준,#수빈,#투모로우바이투게더,#yeonjun,#soobin,#txt,#dra..."
97,",#BTS,#TXT,#ENHYPEN,#HYBE,#HYBEJP_BOYS,#K,#Tak..."
98,",#TOMORROW_X_TOGETHER,#투모로우바이투게더,#TXT,#SOOBIN,..."


In [ ]:
hashtag_data = test.data_hashtagcount()
test.twitterhashtage_to_xlsx(hashtag_data)

Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since


In [ ]:
test.count_wordframe()

C:\Users\Teaw\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,word,count
0,,85
38,#txt,52
35,#tomorrow_x_together,30
44,#we_want_txt_logo,25
45,#where_is_txt7_logo,25
3,#bighit_give_us_the_logo_please,24
75,#투모로우바이투게더,20
48,#you_said_early_may_comeback,15
32,#taehyun,11
73,#태현,10


In [ ]:
count_wordf = test.count_wordframe()
test.twittercount_to_xlsx(count_wordf)

C:\Users\Teaw\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
def tokenize_th(d):
    # print('hey', d)
    d = re.sub('[a-z]', '', d)
    d = re.sub('[A-Z]', '', d)
    d = re.sub('[0-9]', '', d)
    # print('test d', d)
    result = d.split(" ")
    result = list(filter(None, result))
    return result

def clean_text_th(text):
    # text = re.sub('http://\S+|https://\S+', '', text)
    # url = "https://api.aiforthai.in.th/textcleansing"
    # params = {f'text':{text}}
    # headers = {
    #     'Apikey': "REONBZlQhDNoH07AoLRqrOf4dhuAhj9C",
    #     }
    # res = requests.request("GET", url, headers=headers, params=params)
    # # print(res.json())
    # return res.json()['cleansing_text']
    url ='https://api.aiforthai.in.th/lextoplus'
    headers = {'Apikey':"xxxx"}
    
    params = {'text':text,'norm':'1'}
    response = requests.get(url, params=params, headers=headers)
    return response.json()['tokens']

def count_word_en(new_text):
# def count_word_en(dfr):
    # new_text = []
    # for txt in dfr["tweet"]:
    #     new_text.append(test.clean_text(txt))
    
    vectorizer = CountVectorizer(tokenizer=test.tokenize_en)
    transformed_data = vectorizer.fit_transform(new_text)
    keyword_df_en = pd.DataFrame(columns = ['word', 'count'])
    keyword_df_en['word'] = vectorizer.get_feature_names_out()
    keyword_df_en['count'] = np.ravel(transformed_data.sum(axis=0))
    # topword_en = keyword_df_en.sort_values(by=['count'], ascending=False).head(20)

    return keyword_df_en

def count_word_th(new_text):
    # new_text = []
    # for txt in dfr["tweet"]:
    #     new_text.append(test.clean_text(txt))
    
    vectorizer = CountVectorizer(tokenizer = tokenize_th)
    transformed_data = vectorizer.fit_transform(new_text)
    keyword_df_th = pd.DataFrame(columns = ['word', 'count'])
    keyword_df_th['word'] = vectorizer.get_feature_names_out()
    keyword_df_th['count'] = np.ravel(transformed_data.sum(axis=0))
    # topword_en = keyword_df_en.sort_values(by=['count'], ascending=False).head(20)

    return keyword_df_th


dfr = tweets_data
new_text_th = []
new_text_en = []
test_th = ["วันนี้ฝนไม่ตก","พระอาทิตย์แจ่มใส"]
for txt in dfr["tweet"]:
# for txt in test :
    new_text_en.append(test.clean_text(txt))
    # for txt_cut in pyt_wt(txt, keep_whitespace=False,engine="multi_cut"):
    for txt_cut in pyt_wt(txt, keep_whitespace=False,engine="multi_cut"):
    # for txt_cut in clean_text_th(txt):
    #     if txt_cut == '':
    #         pass
    #     else:
            new_text_th.append(txt_cut)
    
    # new_text.append(clean_text_th((txt))) 

# for txt in dfr["tweet"]:
#     new_text_en.append(test.clean_text(txt))
# new_text = new_text_th + new_text_en
# vectorizer = CountVectorizer(tokenizer= tokenize_th)
# vectorizer = CountVectorizer(tokenizer=test.tokenize_en)
# transformed_data = vectorizer.fit_transform(new_text)
# keyword_df_en = pd.DataFrame(columns = ['word', 'count'])
# keyword_df_en['word'] = vectorizer.get_feature_names()
# keyword_df_en['count'] = np.ravel(transformed_data.sum(axis=0))
keyword_df_en = count_word_en(new_text_en)
keyword_df_th = count_word_th(new_text_th)
frames = [keyword_df_en, keyword_df_th]
result = pd.concat(frames)

topword = result.sort_values(by=['count'], ascending=False).head(20)
topword
# vectorizer.get_feature_names_out()

,word,count
262,txt,52
252,tomorrowxtogether,28
298,yeonjun,19
289,whereistxtlogowewanttxtlogobighitgiveusthelogo...,15
268,txtmembers,14
239,taecyj,11
240,taehyun,11
263,txtbighit,10
288,whereistxtlogowewanttxtlogobighitgiveusthelogo...,9
224,soobin,8


In [ ]:
topic_key = "#NCT127"
date_start = "01/04/2022"
date_stop = "07/04/2022"

def formdate(date_start,date_stop):
    form_date = "%d/%m/%Y"
    format_star = datetime.strptime(date_start,form_date)
    format_stop = datetime.strptime(date_stop,form_date)
    diff_deltadays = format_stop - format_star #timedelta
    diff_days = diff_deltadays.days #int
    return format_star,format_stop,diff_days

def cursor_tweets(date_until):
    tweets= tw.Cursor(api.search_tweets, q = topic_key + " -filter:retweets" ,until = date_until).items(100)
    return tweets     
def get_datatweet(tweets):
    tweet_list = []    
    for tweet in tweets:
        tweet_list.append(tweet)
    tweets = tweet_list

    data_tweet = [[topic_key,
                    tweet.user.screen_name,
                    tweet.user.location if tweet.user.location != '' else 'unknown',
                    datetime.strftime(tweet.created_at.replace(tzinfo = None) + timedelta(hours = 7),"%d/%m/%Y"),
                    ClawTwitter.remove_url(tweet.text) if tweet.lang == "en" else ClawTwitter.remove_url_th(tweet.text),
                    tweet.retweet_count,
                    tweet.favorite_count,
                    test.sentiment(TextBlob(test.stemmer(test.clean_text(ClawTwitter.remove_url(tweet.text)))))
                                    if tweet.lang == "en" else test.sentiment_th(test.clean_text_th(tweet.text)),
                    f"https://twitter.com/twitter/statuses/{tweet.id}"] for tweet in tweets]
    return data_tweet

def show_data(data_tweet:list):
    tweet_frame = pd.DataFrame(data=data_tweet,
                        columns=['topic', 'user','location','post date', 'tweet','retweet','likes', 'sentiment','tweet link'])
    return tweet_frame

def twitter_to_xlsx(topic_key,tweets,day_str):
    tweets.to_excel(topic_key+"_"+ day_str+".xlsx", engine="openpyxl", index=False)

def select_datatweet(topic_key,start,stop):
    topic = topic_key
    date_start = start
    date_stop = stop
    format_star,format_stop,diff_days = formdate(date_start,date_stop)
    all_pdtweets = []
    for i in range(diff_days+1):
        date_add = format_star + timedelta(days = int(i))
        date_addstr = date_add.strftime("%Y-%m-%d")
        date_filestr = date_add.strftime("%d%m%Y")
        try:
            print("ok try",date_addstr)
            
            data_frame = pd.read_excel(topic+"_"+date_filestr+".xlsx")
            # readfile = pd.read_excel("#REDVELVET"+"_"+date_opnstr+".xlsx")
            # print(readfile)
        except:
            print("ok except",date_addstr)
            tweets = cursor_tweets(date_addstr)
            data_tweets = get_datatweet(tweets)
            data_frame = show_data(data_tweets)
            twitter_to_xlsx(topic,data_frame,date_filestr)
        all_pdtweets.append(data_frame)
    result = pd.concat(all_pdtweets)
    # result = reduce(lambda left,right: pd.merge(left,right,how='outer'), all_pdtweets)
    # result.to_excel("testreduce"+".xlsx", engine="openpyxl", index=False)
    return result 

select_datatweet(topic_key,date_start,date_stop)

ok try 2022-04-01
ok except 2022-04-01
ok try 2022-04-02
ok except 2022-04-02
ok try 2022-04-03
ok except 2022-04-03
ok try 2022-04-04
ok except 2022-04-04
ok try 2022-04-05
ok except 2022-04-05
ok try 2022-04-06
ok except 2022-04-06
ok try 2022-04-07
ok except 2022-04-07


,topic,user,location,post date,tweet,retweet,likes,sentiment,tweet link
0,#dreamcatcher,urorateez,United States,03/04/2022,please see above DREAMCATCHER DREAMCATCHERApoc...,0,0,neutral,https://twitter.com/twitter/statuses/151040633...
1,#dreamcatcher,NordskovSomnia,Spain • She/her • Esp/Eng,03/04/2022,DreamCatcher hfdreamcatcher,0,0,neutral,https://twitter.com/twitter/statuses/151040598...
2,#dreamcatcher,dcatcherbr,Desde 05/12/2016,03/04/2022,l INFO Na ltima hora BOCA teve mais de mil vis...,13,52,neutral,https://twitter.com/twitter/statuses/151040543...
3,#dreamcatcher,O_N_O_Normani,Somewhere,03/04/2022,Dreamcatcher Heardle 7 Dreamcatcher2ndAlbumApo...,0,0,neutral,https://twitter.com/twitter/statuses/151040469...
4,#dreamcatcher,stonefield_DC,unknown,03/04/2022,New Album Release,0,1,neutral,https://twitter.com/twitter/statuses/151040192...
...,...,...,...,...,...,...,...,...,...
95,#dreamcatcher,wonderjazzbar,unknown,07/04/2022,asianjunkiecom hoping to win Dreamcatcher Apoc...,0,0,positive,https://twitter.com/twitter/statuses/151182194...
96,#dreamcatcher,wonderjazzbar,unknown,07/04/2022,symylife i wouldnt mind any ver im indecisive ...,0,0,neutral,https://twitter.com/twitter/statuses/151182173...
97,#dreamcatcher,DCSupportMX,For Dreamcatcher ♥︎,07/04/2022,Tendencia ApocalypseSaveUs y Dreamcatcher son ...,1,6,neutral,https://twitter.com/twitter/statuses/151182134...
98,#dreamcatcher,mistyteez,unknown,07/04/2022,asianjunkiecom Dreamcatcher ApocalypseSaveUs ty,0,0,neutral,https://twitter.com/twitter/statuses/151182121...
